In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import ood_detector

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.6.2
GPU is NOT AVAILABLE


In [2]:
N_IMAGES = 32
BATCH_SIZE = 16

In [3]:
model = tf.keras.applications.EfficientNetB0()

# ood dataset
x_ood = np.random.normal(loc=0.5, scale=0.2, size=(N_IMAGES, 224, 224, 3))

# id dataset
ds_id = tfds.load('imagenette/160px-v2', split=f'validation[:{N_IMAGES}]', as_supervised=True, try_gcs=True)
ds_id = ds_id.map(
    lambda x, y: (tf.image.resize(tf.image.convert_image_dtype(x, tf.float32), (224, 224))))  # [0, 1]
ds_id = ds_id.map(tf.keras.applications.mobilenet_v3.preprocess_input)
ds_id = ds_id.batch(BATCH_SIZE)
x_id = np.concatenate([a.numpy() for a in ds_id], axis=0)


In [4]:
help(ood_detector)

Help on package ood_detector:

NAME
    ood_detector

PACKAGE CONTENTS
    detectors (package)
    utils (package)

FILE
    /home/tpoppo/Documenti/GitHub/ood-post-hoc/ood_detector/__init__.py




In [7]:
detector = ood_detector.detectors.MSPDetector(model, args_predict={'batch_size': BATCH_SIZE})
detector

In [8]:
score_id = detector.predict(x_id)
score_ood = detector.predict(x_ood)
score_id

array([0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125,
       0.03125, 0.03125, 0.03125, 0.03125], dtype=float32)

In [6]:
score_id = detector.predict(x_id)
score_ood = detector.predict(x_ood)

score_auc = ood_detector.utils.evaluate_aucroc(detector, x_id, x_ood)
score_fpr95 = ood_detector.utils.evaluate_fpr_at_95_tpr(detector, x_id, x_ood)